In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pyspark

import datetime as dt

import tabula
import joblib

In [3]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 60)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import sklearn
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate \
                                    ,cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,recall_score,precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.metrics import roc_curve, auc

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE

In [4]:
passengers = tabula.read_pdf('./data/raw/passengers/usa2017enplanements.pdf',multiple_tables=True, pages='all', guess=False, lattice=True)
df = pd.concat(passengers, axis=0, ignore_index=True)
df = df[[3, 8, 9, 10]]
df = df[1:]
df = df[~df.airport.isin(['Locid'])]
df.dropna(inplace=True)
col = {3: 'airport', 8: 'cy17', 9: 'cy16', 10:'perc_change'}
df.rename(columns=col, inplace=True)

df.cy17 = df.cy17.str.replace(',','')
df.cy16 = df.cy16.str.replace(',','')
df.cy17 = df.cy17.astype('int64')
df.cy16 = df.cy16.astype('int64')

df.perc_change = df.perc_change.str.replace('%','')
df.perc_change = df.perc_change.astype('float64')

In [43]:
# df.to_csv('./data/clean/usa2016-17-enplanements.csv')
# df.to_pickle('./data/clean/usa2016-17-enplanements.pkl')

In [50]:
df.cy17.iloc[0] / df.cy17.sum()

0.05868591831574753

In [45]:
df.head(50)

,airport,cy17,cy16,perc_change
1,ATL,50251962,50501858,-0.490
2,LAX,41232416,39636042,4.030
3,ORD,38593028,37589899,2.670
4,DFW,31816933,31283579,1.700
5,DEN,29809091,28267394,5.450
6,JFK,29533154,29239151,1.010
7,SFO,26900016,25707101,4.640
8,LAS,23364185,22833267,2.330
9,SEA,22639120,21887110,3.440
10,CLT,22011225,21511880,2.320
